### **app.py**
contains routes and functions that define the behavior of the Flask application, handle various URLs, interact with the database, and render different templates based on the requested URLs and submitted form data.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from data import *

app = Flask(__name__)

In [ ]:
# renders the index.html template
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
# renders the devices.html template and passes the brand and smartphones_list as parameters
@app.route('/devices/<brand>')
def devices(brand):
    smartphones_list = read_smartphones_by_brand(brand)
    return render_template('devices.html',brand=brand, smartphones=smartphones_list)

In [ ]:
# renders the smartphone.html template and passes the smartphone as a parameter
@app.route('/devices/<int:smartphone_id>')
def smartphone(smartphone_id):
    smartphone = read_smartphone_by_id(smartphone_id)
    return render_template('smartphone.html',smartphone=smartphone)

In [ ]:
# renders the register.html template
@app.route('/register')
def register():
    return render_template('register.html')

In [ ]:
# processes the form data submitted via POST method and inserts a new smartphone into the database
@app.route('/processed', methods=['POST'])
def processing():
    smartphone_data = {
        "brand": request.form['brand'],
        "model": request.form['brand_model'],
        "chipset": request.form['brand_chipset'],
        "main_camera": request.form['brand_main_camera'],
        "selfie_camera": request.form['brand_selfie_camera'],
        "internal_memory": request.form['brand_internal_memory'],
        "battery": request.form['brand_battery'],
        "color": request.form['brand_color'],
        "price": request.form['brand_price'],
        "condition": request.form['brand_condition'],
        "reason_for_selling": request.form['brand_reason_for_selling'],
        "url": request.form['brand_url']
    }
    insert_smartphone(smartphone_data)
    return redirect(url_for('devices', brand=request.form['brand']))

In [ ]:
# processes the form data submitted via POST method and performs an action based on the "modify" value
@app.route('/modify', methods=['post'])
def modify():
 if request.form["modify"] == "edit":
    smartphone_id = request.form["smartphone_id"]
    smartphone = read_smartphone_by_id(smartphone_id)
    return render_template('update.html', smartphone=smartphone)
 elif request.form["modify"] == "delete":
    smartphone_id =request.form["smartphone_id"]
    smartphone = read_smartphone_by_id(smartphone_id)
    delete_smartphone(smartphone_id)
    return redirect(url_for("devices", brand=smartphone['brand']))

In [ ]:
# processes the form data submitted via POST method and updates the smartphone details in the database
@app.route('/update', methods=['post'])
def update():
    smartphone_data = {
        "id": request.form["smartphone_id"],
        "brand": request.form['smartphone_brand'],
        "model": request.form['smartphone_model'],
        "chipset": request.form['smartphone_chipset'],
        "main_camera": request.form['smartphone_main_camera'],
        "selfie_camera": request.form['smartphone_selfie_camera'],
        "internal_memory": request.form['smartphone_internal_memory'],
        "battery": request.form['smartphone_battery'],
        "color": request.form['smartphone_color'],
        "price": request.form['smartphone_price'],
        "condition": request.form['smartphone_condition'],
        "reason_for_selling": request.form['smartphone_reason_for_selling'],
        "url": request.form['smartphone_url']
    }
    update_smartphone(smartphone_data)
    return redirect(url_for('smartphone', smartphone_id = request.form['smartphone_id']))

In [ ]:
# processes the search query submitted via GET method and retrieves the search results from the database
@app.route('/search', methods=['get'])
def search():
    query = request.args.get('query', '')
    results = search_smartphones(query)
    return render_template('search_results.html', query=query, results=results)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

### **data.py**
provides basic database operations for interacting with the SQLite database and performing CRUD (Create, Read, Update, Delete) operations on the smartphone records.

In [ ]:
# establishes a connection to the SQLite database at the specified path and returns the connection and cursor objects
def connect_to_db(path):
    conn = sqlite3.connect(path)
    conn.row_factory = sqlite3.Row
    return (conn, conn.cursor())

In [ ]:
# retrieves a list of smartphones filtered by the specified brand from the database
def read_smartphones_by_brand(brand):
    conn, cur = connect_to_db(db_path)
    query = 'SELECT * FROM smartphones WHERE brand = ?'
    value = brand
    results = cur.execute(query,(value,)).fetchall()
    conn.close()
    return results

In [ ]:
# retrieves a list of smartphones filtered by the specified brand from the database
def read_smartphone_by_id(id):
    conn, cur = connect_to_db(db_path)
    query = 'SELECT * FROM smartphones WHERE id = ?'
    value = id
    results = cur.execute(query,(value,)).fetchone()
    conn.close()
    return results

In [ ]:
# inserts a new smartphone record into the database
def insert_smartphone(smartphone_data):
    conn, cur = connect_to_db(db_path)
    query = 'INSERT INTO smartphones (brand, model, chipset, main_camera, selfie_camera, internal_memory, battery, color, price, condition, reason_for_selling, url) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'
    values = (smartphone_data['brand'], smartphone_data['model'],
              smartphone_data['chipset'], smartphone_data['main_camera'],
              smartphone_data['selfie_camera'], smartphone_data['internal_memory'],
              smartphone_data['battery'], smartphone_data['color'],
              smartphone_data['price'], smartphone_data['condition'],
              smartphone_data['reason_for_selling'], smartphone_data['url'])
    cur.execute(query,values)
    conn.commit()
    conn.close()

In [ ]:
# updates an existing smartphone record in the database
def update_smartphone(smartphone_data):
    conn, cur = connect_to_db(db_path)
    query = "UPDATE smartphones SET brand=?, model=?, chipset=?, main_camera=?, selfie_camera=?, internal_memory=?, battery=?, color=?, price=?, condition=?, reason_for_selling=?, url=? WHERE id=?"
    values = (smartphone_data['brand'], smartphone_data['model'],
              smartphone_data['chipset'], smartphone_data['main_camera'],
              smartphone_data['selfie_camera'], smartphone_data['internal_memory'],
              smartphone_data['battery'], smartphone_data['color'],
              smartphone_data['price'], smartphone_data['condition'],
              smartphone_data['reason_for_selling'], smartphone_data['url'],
              smartphone_data['id'])
    cur.execute(query,values)
    conn.commit()
    conn.close()

In [ ]:
# deletes a smartphone record from the database based on the specified ID
def delete_smartphone(smartphone_id):
    conn, cur = connect_to_db(db_path)
    query = "DELETE FROM smartphones WHERE id = ?"
    values = (smartphone_id,)
    cur.execute(query, values)
    conn.commit()
    conn.close()

In [ ]:
# searches for smartphones in the database based on the specified query
def search_smartphones(query):
    conn, cur = connect_to_db(db_path)
    sql_query = "SELECT * FROM smartphones WHERE brand LIKE ? OR model LIKE ?"
    value = "%{}%".format(query)
    results = cur.execute(sql_query, (value, value)).fetchall()
    conn.close()
    return results

### **base.html**
serves as the base template for other pages in the Greenhills Online Extension website.

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" crossorigin="anonymous">
    <link rel="stylesheet" href="{{ url_for('static',filename = 'css/style.css') }}">
    <title>Greenhills Online Extension</title>
</head>
<body>

# allows reusing the same navigation bar across different pages
    {% include 'nav.html' %}

    <div class="m-4">
# defines the content block that can be overridden in child templates
        {% block content %}
        <h1>This is where the content is</h1>
        {% endblock %}
    </div>


    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js" integrity="sha384-geWF76RCwLtnZ8qwWowPQNguL3RmwHVBC9FhGdlKrxdiJJigb/j/68SIy3Te4Bkz" crossorigin="anonymous"></script>
</body>
</html>

### **devices.html**
generates a page that displays a list of smartphones belonging to a specific brand and for each smartphone, its model is displayed as a clickable link that leads to the corresponding details page.

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
<h1>List of {{ brand }}</h1>
<ul>
# defines a loop that iterates over a list of smartphones then retrieves each object from the list one by one
    {% for smartphone in smartphones %}
    <li><a href="/devices/{{ smartphone.id }}">{{ smartphone.model }}</a></li>
    {% endfor %}
</ul>
# marks the end of the {% block content %}
{% endblock %}

### **index.html**
provides a simple homepage for the Greenhills Reseller Website Extension with a heading, a description, and a list of smartphone brands as clickable links, allowing users to navigate to the pages displaying devices from the selected brands.

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
<h1>Greenhills Reseller Website Extension</h1>
<p>Search through the list of smartphone brands to find a good and affordable smartphone!</p>
<ul>
# creates a list of hyperlinks that lead to the pages displaying devices from the respective smartphone brands: Apple, Samsung, and Xiaomi
    <li><a href="devices/Apple">Apple</a></li>
    <li><a href="devices/Samsung">Samsung</a></li>
    <li><a href="devices/Xiaomi">Xiaomi</a></li>
</ul>
# marks the end of the {% block content %}
{% endblock %}

### **nav.html**
represents a navigation bar implemented using the Bootstrap framework, which is designed to be responsive, expanding and collapsing on smaller screens.

In [ ]:
# represents a navigation bar using the Bootstrap framework, designed to expand and collapse on smaller screens
<nav class="navbar navbar-expand-lg navbar-dark bg-success">
# creates a container to hold the navbar content
  <div class="container-fluid">
    <a class="navbar-brand" href="/">Greenhills Online Extension</a>
# allows the navbar to collapse and expand when clicked
    <button class="navbar-toggler" type="button" data-bs-toggle="collapse" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
      <span class="navbar-toggler-icon"></span>
    </button>
# contains the navbar items that will collapse or expand based on the toggle button's state
    <div class="collapse navbar-collapse" id="navbarSupportedContent">
# represents the navbar's menu items
      <ul class="navbar-nav me-auto mb-2 mb-lg-0">
        <li class="nav-item">
          <a class="nav-link active" aria-current="page" href="/">Home</a>
        </li>
        <li class="nav-item">
          <a class="nav-link" href="{{ url_for('register') }}">Post Your Item!</a>
        </li>
      </ul>
# represents the search functionality in the navbar
      <form class="d-flex" action="{{ url_for('search') }}" method="GET">
        <input class="form-control me-2" type="search" name="query" placeholder="Type something here" aria-label="Search">
        <button class="btn btn-outline-success text-light" type="submit">Go!</button>
      </form>
    </div>
  </div>
</nav>

### **register.html**
contains a form for users to post details of their preloved smartphones for potential buyers. The form data is submitted to the specified URL using the HTTP POST method for processing.

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
<h1>Post your preloved smartphones here for potential buyers!</h1>
# submits data to the specified URL using the HTTP POST method for processing
<form action="{{ url_for('processing') }}" method="post">
# represents a dropdown list that allows the user to select a brand from the provided options then assigns the selected value to the "brand" attribute when the form is submitted
    <label for="p_brand">Smartphone Brand:</label>
    <select name="brand" id="p_brand" required>
        <option disabled selected value>...</option>
        <option value="Apple">Apple</option>
        <option value="Samsung">Samsung</option>
        <option value="Xiaomi">Xiaomi</option>
    </select><br><br>
# associated with an input field where the user can enter a value, and the entered value will be assigned to an attribute when the form is submitted. The 'required' attribute means it must be filled in before the form can be submitted
    <label for="p_model">Model:</label>
    <input type="text" name="brand_model" id="p_model" required><br><br>
    <label for="p_chipset">Chipset:</label>
    <input type="text" name="brand_chipset" id="p_chipset"><br><br>
    <label for="p_main_camera">Main Camera:</label>
    <input type="text" name="brand_main_camera" id="p_main_camera"><br><br>
    <label for="p_selfie_camera">Selfie Camera:</label>
    <input type="text" name="brand_selfie_camera" id="p_selfie_camera"><br><br>
    <label for="p_internal_memory">Internal Memory:</label>
    <input type="text" name="brand_internal_memory" id="p_internal_memory"><br><br>
    <label for="p_battery">Battery:</label>
    <input type="text" name="brand_battery" id="p_battery"><br><br>
    <label for="p_color">Color:</label>
    <input type="text" name="brand_color" id="p_color"><br><br>
    <label for="p_price">Price:</label>
    <input type="number" name="brand_price" id="p_price"><br><br>
    <label for="p_condition">Condition:</label>
    <textarea name="brand_condition" id="p_condition" placeholder="What is the condition of your smartphone?" cols="30" rows="10"></textarea><br><br>
    <label for="p_reason_for_selling">Reason For Selling:</label>
    <textarea name="brand_reason_for_selling" id="p_reason_for_selling" placeholder="Why do you want to sell this?" cols="30" rows="10"></textarea><br><br>
    <label for="p_url">Image URL:</label>
    <input type="url" name="brand_url" id="p_url"><br><br>
# represents a submit button that, when clicked, submits the form data to the server for processing with the label "Register!" displayed as the button text
    <input type="submit" value="Register!">
</form>
# marks the end of the {% block content %}
{% endblock %}

### **search_results.html**
holds search results for a given query, and iterates over each smartphone object in the results list to display its brand, model, image, and various details. If no results are found, it displays a message indicating no results.

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
  <h1>Search Results for "{{ query }}"</h1>
# checks if there are any search results available
  {% if results %}
    <ul>
# starts a loop that iterates over each item in the results list, allowing access to individual smartphone objects within the loop
      {% for smartphone in results %}
        <li>
# displays the brand and model of the smartphone using the brand and model attributes of the smartphone object
          <h2>{{ smartphone.brand }} - {{ smartphone.model }}</h2>
# displays an image of the smartphone using the url attribute of the smartphone object
          <img src="{{ smartphone.url }}" alt="{{ smartphone.brand }} {{ smartphone.model }}">
# displays various details of the smartphone, using the corresponding attribute of the object
          <p>Chipset: {{ smartphone.chipset }}</p>
          <p>Main Camera: {{ smartphone.main_camera }}</p>
          <p>Selfie Camera: {{ smartphone.selfie_camera }}</p>
          <p>Internal Memory: {{ smartphone.internal_memory }}</p>
          <p>Battery: {{ smartphone.battery }}</p>
          <p>Color: {{ smartphone.color }}</p>
          <p>Price: {{ smartphone.price }}</p>
          <p>Condition: {{ smartphone.condition }}</p>
          <p>Reason for Selling: {{ smartphone.reason_for_selling }}</p>
        </li>
      {% endfor %}
    </ul>
# executes the code block below (display "No results found.") if there are no search results
  {% else %}
    <p>No results found.</p>
  {% endif %}
# marks the end of the {% block content %}
{% endblock %}

### **smartphone.html**
contains details of each specific smartphone object. It also provides a form with "edit" and "delete" buttons for modifying or deleting the smartphone object.

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
<h1> {{ smartphone.model }} </h1>
<img src="{{smartphone.url}}">
# displays various attributes of a smartphone object
<ul>
    <li>Chipset: {{ smartphone.chipset }}</li>
    <li>Main Camera: {{ smartphone.main_camera }}</li>
    <li>Selfie Camera: {{ smartphone.selfie_camera }}</li>
    <li>Internal_Memory: {{ smartphone.internal_memory }}</li>
    <li>Battery: {{ smartphone.battery }}</li>
    <li>Price: {{ smartphone.price }}</li>
    <li>Condition: {{ smartphone.condition }}</li>
    <li>Reason for Selling: {{ smartphone.reason_for_selling }}</li>
</ul>
# generates a form with two submit buttons, "edit" and "delete," which are used for modifying or deleting a specific smartphone object
<form action="{{ url_for('modify') }}" method="post">
    <input type="hidden" name="smartphone_id" value="{{ smartphone.id }}">
    <input class="btn btn-secondary" type="submit" name="modify" value="edit"></input>
    <input class="btn btn-danger" type="submit" name="modify" value="delete"></input>
</form>
# marks the end of the {% block content %}
{% endblock %}

### **update.html**
a form for editing phone details, and provides a submit button to trigger an update action, allowing changes to be made and saved

In [ ]:
# extends the base.html template to inherit the structure and styling defined in the base template
{% extends 'base.html' %}

# overrides the block in base.html and the content within will replace the default content in base.html
{% block content %}
    <h1>Edit phone details</h1>
# creates an HTML form that will send a POST request to the "update" endpoint when submitted, allowing for data updates or modifications
    <form action="{{ url_for('update') }}" method="post">
        <input type="hidden" name="smartphone_id" value="{{ smartphone.id }}">
        <label for="p_name">Brand:</label>
        <input type="text" name="smartphone_brand" id="p_name" value="{{ smartphone.brand }}" required><br><br>
        <label for="p_model">Model:</label>
        <input type="text" name="smartphone_model" id="p_model" value="{{ smartphone.model }}"><br><br>
        <label for="p_chipset">Chipset:</label>
        <input type="text" name="smartphone_chipset" id="p_chipset" value="{{ smartphone.chipset }}"><br><br>
        <label for="p_main_camera">Main Camera:</label>
        <input type="text" name="smartphone_main_camera" id="p_main_camera" value="{{ smartphone.main_camera }}"><br><br>
        <label for="p_selfie_camera">Selfie Camera:</label>
        <input type="text" name="smartphone_selfie_camera" id="p_selfie_camera" value="{{ smartphone.selfie_camera }}"><br><br>
        <label for="p_internal_memory">Internal Memory:</label>
        <input type="text" name="smartphone_internal_memory" id="p_internal_memory" value="{{ smartphone.internal_memory }}"><br><br>
        <label for="p_battery">Battery:</label>
        <input type="text" name="smartphone_battery" id="p_battery" value="{{ smartphone.battery }}"><br><br>
        <label for="p_color">Color:</label>
        <input type="text" name="smartphone_color" id="p_color" value="{{ smartphone.color }}"><br><br>
        <label for="p_price">Price:</label>
        <input type="number" name="smartphone_price" id="p_price" value="{{ smartphone.price }}"><br><br>
        <label for="p_condition">Condition:</label>
        <input type="text" name="smartphone_condition" id="p_condition" value="{{ smartphone.condition }}"><br><br>
        <label for="p_reason_for_selling">Reason For Selling:</label>
        <textarea name="smartphone_reason_for_selling" cols="30" rows="10">{{ smartphone.reason_for_selling }}</textarea><br><br>
        <label for="p_url">URL:</label>
        <input type="url" name="smartphone_url" id="p_url" value="{{ smartphone.url }}"><br><br>
# generates a submit button within a form that triggers an update action when clicked, typically used to update or save changes made to a form
        <input type="submit" value="Update!">
    </form>
# marks the end of the {% block content %}
{% endblock %}